In [3]:
# Instalação de libs
!pip install requests
!pip install boto3
!pip install python-dotenv
!pip install SQLAlchemy
!pip install psycopg2-binary
!pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 34.8 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 39.3 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [4]:
# Importar biblioteca completa
import requests
import zipfile
import io
import pandas as pd
import os
import boto3
import sys
import time 

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from botocore.exceptions import BotoCoreError, ClientError
from psycopg2 import sql

In [5]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("-", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)
        sys.exit()

In [6]:
# Calcular o tempo de execução
def calcular_tempo_execucao(tempo_inicio, tempo_final):
    """
    Calcula e exibe o tempo total de execução de um processo.

    Parâmetros:
    tempo_inicio (float): O tempo de início capturado com time.time()
    tempo_final (float): O tempo final capturado com time.time()

    A função não retorna nada, apenas imprime o resultado formatado.
    """
    # Calcula a diferença total em segundos
    tempo_total_segundos = tempo_final - tempo_inicio

    # Converte o total de segundos para minutos e segundos
    minutos, segundos = divmod(tempo_total_segundos, 60)

    # Imprime o resultado
    print(f"\nTempo total de execução do código:")
    print(f"{int(minutos)} minutos e {int(segundos)} segundos")

In [ ]:
# Nome do bucket S3 e subpastas
s3_bucket = 'fiaptechchallengefase3'
s3_subpasta_bronze = 'bronze'
s3_subpasta_silver = 'silver'
s3_subpasta_gold = 'gold'

silver_prefix = f's3://{s3_bucket}/{s3_subpasta_silver}/'
gold_prefix = f's3://{s3_bucket}/{s3_subpasta_gold}/'

# Nome arquivo .parquet
nome_arquivo_gold = 'pnad_final_tratado.parquet'

# Caminho de saida e entrada camada silver
caminho_saida_silver = silver_prefix + 'pnad_consolidado_enriquecido.parquet'
caminho_entrada_silver = silver_prefix + 'pnad_consolidado_enriquecido.parquet'
caminho_saida_gold = gold_prefix + 'pnad_final_tratado.parquet'
caminho_completo_gold = gold_prefix + nome_arquivo_gold

# Caminho do Github com dados do PNAD
api_url = 'https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados'

# Caminho do Github com dados do código IBGE UF
link_codigo_uf = 'https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv'

# Nome da tabela no PostgreSQL
nome_tabela = 'questionario_pnad_covid'